In [1]:
import pandas as pd
pd.options.display.max_rows = 999
import numpy as np
import time
import datetime as dt
import os
import re
import sys
import string

#sklearn
# from sklearn.cross_validation import train_test_split
from sklearn import metrics

from custom_transformer import ColumnSelector, ApplyFunction, ColumnTransform, PrintThat
import custom_transformer as ct

In [2]:
dataset=pd.read_csv("ticket_all_lev.csv")

In [ ]:
dataset

In [3]:
dataset=dataset[np.isfinite(dataset['tpl_id'])]

In [ ]:
datasetSorted = dataset.sort_values(["tpl_id"],ascending=[1])

In [5]:
datasetSorted2 = dataset.sort_values(["ticket_category"],ascending=[1])

In [6]:
datasetSorted2

,create_by,highest_tpl_id,is_cs,is_recommendation,reply,reply_create_time,ticket_category,ticket_create_time,ticket_id,ticket_msg,ticket_title,tpl_id,tpl_n_ratio,user_id
8230,3044829,95,1,NaN,"Hai.\r\n\r\nBerdasarkan pengecekan kami, dana ...",2016-02-18 17:03:53.576600,1,NaN,258973,Saya udah mengisi semua data pada konfirmasi p...,konfirmasi pembelian,95.0,95[0.983561643836] |,1904651
11123,669010,446,1,NaN,Terima kasih atas dukungan dan kepercayaan And...,2016-01-25 21:28:01.537311,1,NaN,174953,https://www.tokopedia.com/winsshopsolo/info to...,Pelanggaran merk logo dan foto,446.0,446[0.987854251012] |,936319
8769,344613,439,1,NaN,"Hi Herry,\r\n\r\nMaaf atas ketidaknyamanannya....",2016-02-15 05:56:14.137005,1,NaN,207530,pada tanggal 06/12/14 saya membeli hp sony xpe...,penipuan,439.0,439[0.85296803653] | 78[0.805270863836] |,1934145
7844,7000832,281,1,NaN,"Hai Siti,\r\n\r\nMohon maaf atas ketidaknyaman...",2016-02-21 14:42:48.154389,1,NaN,337869,Ka biasanya barang diproses oleh penjual berap...,barang,281.0,281[0.859764089122] |,3825890
7602,7575044,516,1,NaN,"Hi Kisshops,\r\n\r\nDapat kami informasikan, G...",2016-02-23 10:26:37.330676,1,NaN,332074,Selamat Pagi\r\nSaya KISSOPS salah satu toko y...,Cara untuk Produk kami berada di Halaman Pertama,516.0,516[0.988298829883] |,535093
1767,6622461,509,1,1.0,Hai \r\n\r\nSebelumnya mohon maaf atas ketidak...,2016-03-24 09:35:51.627844,1,NaN,357427,Mohon.untuk invoice INV/20150721/XV/VII/149130...,kelebihan bayar,509.0,509[1.0] |,1635405
9638,4657184,504,1,NaN,"Hallo\r\n\r\nUntuk permintaan penarikan dana, ...",2016-02-08 07:09:56.180786,1,NaN,332661,knp semua dana aku yang sdh cair di review?pad...,tanya,504.0,504[0.939759036145] |,3217968
7054,3464225,281,1,NaN,"Hai\r\n\r\nBerdasarkan pengecekan, status peme...",2016-02-26 15:57:28.192514,1,NaN,337869,Ka biasanya barang diproses oleh penjual berap...,barang,281.0,281[0.983341746593] |,3825890
9667,3464225,503,1,NaN,"Halo,\r\n\r\nSaat ini untuk pembayaran Anda ma...",2016-02-07 20:08:16.443441,1,NaN,227883,Lap pel super mop..saya sudah transfer melalui...,sdh transfer blm ad konfirmasi,503.0,503[0.81558676937] |,1584994
5385,5974692,448,1,1.0,"Halo Wisa,\r\n\r\nMohon maaf, dikarenakan adan...",2016-03-07 13:14:02.961647,1,NaN,350653,"selamat siang saya mau minta bantuan , ini tra...",tidak bisa input resi,448.0,448[0.939351198872] |,2466660


In [ ]:
ds_X = datasetSorted.ix[:,["ticket_title","ticket_msg","ticket_category","tpl_id"]]

In [ ]:
ds_y = datasetSorted["tpl_id"]

In [ ]:
XTrain,XTest,yTrain,yTest = ct.train_test_split(ds_X,ds_y,test_size=0.5,random_state=7)

In [ ]:
pipe = ct.Pipeline([
                ("features", ct.FeatureUnion([
                            ("message", ct.Pipeline([
                                        ("select_text", ColumnSelector("ticket_msg")),       
#                                         ("debug1", ct.PrintThat()),
                                        ("preprocessing_input", ApplyFunction(ct.preprocess_input)),
                                        ("delete_link", ApplyFunction(ct.clean_link)),
                                        ("clean_text", ApplyFunction(ct.clean_symbol)),
                                        ("hashing_vectorizer", ct.HashingVectorizer()),
                                    ])
                            ),
                            ("title", ct.Pipeline([
                                        ("select_text", ColumnSelector("ticket_title")),                                        
                                        ("preprocessing_input", ApplyFunction(ct.preprocess_input)),
                                        ("take_link", ApplyFunction(ct.take_link)),
                                        ("clean_link", ApplyFunction(ct.clean_symbol)),
                                        ("hashing_vectorizer", ct.HashingVectorizer()),
#                                         ("debug2", PrintThat())
                                    ])
                            ),
                            ("category", ColumnSelector("ticket_category")),
 
                        ], transformer_weights={"message":1, "title":1,"category":1})
                ),

                ("classifier", ct.LinearSVC())

            ])

In [ ]:
a=ColumnSelector("ticket_category")
XTrain[a]

In [ ]:
start_time = time.time()
print type(XTrain)
print type(yTrain)

pipe.fit(XTrain, yTrain)

p_time = time.time() - start_time
print("predict time is {}".format(p_time))

In [ ]:
y_predict=pipe.predict(XTest)

In [ ]:
y_predict

In [ ]:
#     cross validation
start_time = time.time()
cv_score = ct.cross_val_score(pipe, XTrain, yTrain, cv=5)
cv_time = time.time() - start_time
print("predict time is {}".format(cv_time))

print("CV accuracy: %0.2f (+/- %0.2f)" % (cv_score.mean(), cv_score.std() * 2))